# Soft Computing

## Cvičení 12 - Bayesovské sítě

### On-site cvičení

#### 12.1 Závislost proměnných

Naivní Bayesův klasifikátor je založen na předpokladu, že všechny náhodné jevy jsou na sobě nezávislé, což reálnách situacích není převážně pravdou. Pro tyto reálné problémy budeme potřebovat jiné typy modelů, které umožňují specifikovat různorodé typy náhodných proměnných:
1. Vzájemně nezávislé
2. Vzájemně nezávislé
3. Vzájemně podmíněně nezávislé

Bayesovské sítě jsou typ pravděpodobnostního grafického modelu, který je matematicky reprezentován acyklickým orientovaným grafem. Uzly grafu představují pravděpodobnostní proměnné a hrany mezi nimi představují podmíněnou vzájemnou závislost. Svou vyjadřovací silou jsou schopny vyjádřit všechny tři typy pravděpodobnostních proměnných.

<img src="SOC12_soubory/promenne.png" style="height:250px">

V tomto pravděpodobnostním grafickém modelu vidíme:
* **Vzájemně nezávislé proměnné:** A-C a B-C, zda nastane událost C nemá vliv na událost A a opačně
* **Vzájemně závislé proměnné:** A-B, B je potomek A a je tedy na něm existenčně závislý podmíněnou pravděpodobností
* **Vzájemně podmíněné nezávislé:** A-D, D je podmíněně nezávislý na A, jelikož je jeho prarodičem a vytváří závislost pro D díky svému potomkovi B



#### 12.2 Bayesovské sítě (Bayesian Belief Network, BBN)

Pro tvorbu BBN slouží v Pythonu modul s názvem PyBBN.

#### 12.3 Předpověd počasí pomocí BBN

Aplikujme BBN na vybranou datovou sadu z Kaggle. Na Kaggle se nachází datová sada o Australském počasí, která nám umožní predikovat, zda bude zítra pršet nebo ne.

Odkaz na dataset: [ZDE](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package)

<img src="SOC12_soubory/pocasi.png" style="height:350px">

**Import modulů**

In [ ]:
import pandas as pd                 # pro manipulaci s daty
import networkx as nx               # pro vykreslování grafů
import matplotlib.pyplot as plt     # pro vykreslování grafů

# pro tvorbu BBN modelu
from pybbn.graph.dag import Bbn
from pybbn.graph.edge import Edge, EdgeType
from pybbn.graph.jointree import EvidenceBuilder
from pybbn.graph.node import BbnNode
from pybbn.graph.variable import Variable
from pybbn.pptc.inferencecontroller import InferenceController

**Stažení datové sady**

In [ ]:
!wget https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

**Analýza dat**

In [ ]:
# nastavíme v PD, abychom si mohli prohlédnout více sloupců než implicitně modul ukazuje
pd.options.display.max_columns=50

# načtení dat z csv souboru do pandas datového rámce
df=pd.read_csv('weatherAUS.csv', encoding='utf-8')

# zahození záznamů, ve kterých chybí příznak, zda druhý den pršelo (cíl klasifikace)
df=df[pd.isnull(df['RainTomorrow'])==False]

# pokud někde jinde chybí data, tak je vyplň střední hodnotou
df=df.fillna(df.mean())

# Create bands for variables that we want to use in the model
df['WindGustSpeedCat']=df['WindGustSpeed'].apply(
    lambda x: '0.<=40' if x<=40 else '1.40-50' if 40<x<=50 else '2.>50'
)

#
df['Humidity9amCat']=df['Humidity9am'].apply(
    lambda x: '1.>60' if x>60 else '0.<=60'
)

#
df['Humidity3pmCat']=df['Humidity3pm'].apply(
    lambda x: '1.>60' if x>60 else '0.<=60'
)

# prohlédnutí výsledného datového rámce
df

**Konstrukce uzlů sítě**

Vytvoříme uzly BBN sítě. Pravděpodobnosti do BBN uzlů se získají pomocí normalizované frekvence výskytu hodnot pravděpodobností proměnné.

In [ ]:
df['Humidity9amCat'].value_counts().sort_index()

In [ ]:
df['Humidity9amCat'].value_counts(normalize=True).sort_index()

In [ ]:
H9am = BbnNode(Variable(0, 'H9am', ['<=60', '>60']), [0.30658, 0.69342])

Proměnné, které mají rodiče, musí mít specifikovány pravděpodobnosti pro všechny kombinace hodnot.

<img src="SOC12_soubory/rodic.png" style="height:100px">

In [ ]:
H3pm = BbnNode(Variable(1, 'H3pm', ['<=60', '>60']), [0.92827, 0.07173, 
                                                      0.55760, 0.44240])

In [ ]:
W = BbnNode(Variable(2, 'W', ['<=40', '40-50', '>50']), [0.58660, 0.24040, 0.17300])

In [ ]:
RT = BbnNode(Variable(3, 'RT', ['No', 'Yes']), [0.92314, 0.07686, 
                                                0.89072, 0.10928, 
                                                0.76008, 0.23992, 
                                                0.64250, 0.35750, 
                                                0.49168, 0.50832, 
                                                0.32182, 0.67818])

Výpočet pravděpodobností manuálně je nevhodná metoda pro tvorbu větších sítí a navíc zavádí lidský chybový faktor. Vytvoříme proměnné a jejich podmíněné pravděpodobnosti pomocí pomocné funkce.

In [ ]:
# tato funkce pomáhá spočítat podmíněné pravděpodobnosti (funguje uzly s nejvyyším počtem rodičů 2)
def probs(data, child, parent1=None, parent2=None):
    #pokud uzel nemá rodiče
    if parent1==None:
        prob=pd.crosstab(data[child], 'Empty', margins=False, normalize='columns').sort_index().to_numpy().reshape(-1).tolist()
    # pokud má uzel jednoho rodiče
    elif parent1!=None and parent2==None:
        prob=pd.crosstab(data[parent1],data[child], margins=False, normalize='index').sort_index().to_numpy().reshape(-1).tolist()
    elif parent1!=None and parent2!=None:    
        # pokud má uzel dva rodiče
        prob=pd.crosstab([data[parent1],data[parent2]],data[child], margins=False, normalize='index').sort_index().to_numpy().reshape(-1).tolist()
    else: 
        print("Error in Probability Frequency Calculations")
    return prob 

In [ ]:
# tvorba uzlů pomocí pomocné funkce
H9am = BbnNode(Variable(0, 'H9am', ['<=60', '>60']), probs(df, child='Humidity9amCat'))
H3pm = BbnNode(Variable(1, 'H3pm', ['<=60', '>60']), probs(df, child='Humidity3pmCat', parent1='Humidity9amCat'))
W = BbnNode(Variable(2, 'W', ['<=40', '40-50', '>50']), probs(df, child='WindGustSpeedCat'))
RT = BbnNode(Variable(3, 'RT', ['No', 'Yes']), probs(df, child='RainTomorrow', parent1='Humidity3pmCat', parent2='WindGustSpeedCat'))

**Konstrukce modelu BBN**

In [ ]:
# tvorba modelu sítě
bbn = Bbn() \
    .add_node(H9am) \
    .add_node(H3pm) \
    .add_edge(Edge(H9am, H3pm, EdgeType.DIRECTED)) \
    .add_node(RT) \
    .add_edge(Edge(H3pm, RT, EdgeType.DIRECTED)) \
    .add_node(W) \
    .add_edge(Edge(W, RT, EdgeType.DIRECTED))

# převedení BNN do stromu
join_tree = InferenceController.apply(bbn)

**Vykreslení BBN**

In [ ]:
# nastavení pozic uzlů
pos = {
    0: (-1, 2), 
    1: (-1, 0.5), 
    2: (1, 0.5), 
    3: (0, -1)
}

# nastavení vzhledu grafu
options = {
    "font_size": 16,
    "node_size": 4000,
    "node_color": "white",
    "edgecolors": "black",
    "edge_color": "red",
    "linewidths": 5,
    "width": 5,}
    
# generování grafu
n, d = bbn.to_nx_graph()
nx.draw(n, with_labels=True, labels=d, pos=pos, **options)

# tisk grafu
ax = plt.gca()
ax.margins(0.10)
plt.axis("off")
plt.show()

**Predikce**

In [ ]:
# Define a function for printing marginal probabilities
def print_probs():
    for node in join_tree.get_bbn_nodes():
        potential = join_tree.get_bbn_potential(node)
        print("Node:", node)
        print("Values:")
        print(potential)
        print('----------------')
        
# Use the above function to print marginal probabilities
print_probs()

**Dodání důkazů do BBN**

In [ ]:
# při dodání důkazů musí být pravděpodobnosti přepočítány
def evidence(ev, nod, cat, val):
    ev = EvidenceBuilder() \
    .with_node(join_tree.get_bbn_node_by_name(nod)) \
    .with_evidence(cat, val) \
    .build()
    join_tree.set_observation(ev)
    
# přidej důkaz
evidence('ev1', 'H9am', '>60', 1.0)

# vytiskni pravděpodobnosti
print_probs()